<a href="https://colab.research.google.com/github/Anitha0120/MULTILINGUAL-VIDEO-DUBBING/blob/main/MINOR_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googletrans==4.0.0-rc1 # Install or update googletrans package
!pip install openai-whisper
!pip install gtts
!pip install pydub # Install the pydub library.
!pip install pysrt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=beea26a553af8eb838e1708eb322c219fef24041bd0f7079aa4e371c3f195e26
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [ ]:
import os
import whisper
from moviepy.editor import VideoFileClip, AudioFileClip
from googletrans import Translator
from gtts import gTTS
from pydub import AudioSegment
import pysrt

# ==== CONFIGURATION ====
video_path = "/content/input.mp4"
translated_video_path = "translated_video.mp4"
translated_audio_path = "translated_audio.mp3"
translated_srt_path = "translated_subs.srt"
temp_audio_path = "temp_audio.wav"
chosen_language = "malayalam"  # Change to your target language

# ==== Language Support ====
language_codes = {
    "english": "en", "hindi": "hi", "bengali": "bn", "tamil": "ta", "telugu": "te", "marathi": "mr",
    "gujarati": "gu", "kannada": "kn", "malayalam": "ml", "punjabi": "pa", "urdu": "ur",
    "assamese": "as", "maithili": "mai", "santali": "sat", "bodo": "brx", "dogri": "doi",
    "kashmiri": "ks", "konkani": "gom", "sanskrit": "sa", "sindhi": "sd"
}
gtts_supported = {"hi", "bn", "ta", "te", "mr", "gu", "kn", "ml", "pa", "ur"}

lang_code = language_codes.get(chosen_language.lower(), "ml")
use_tts = lang_code in gtts_supported

# ==== Step 1: Extract audio from video ====
print("🔊 Extracting audio from video...")
video = VideoFileClip(video_path)
video.audio.write_audiofile(temp_audio_path)

# ==== Step 2: Transcribe English audio ====
print("🧠 Transcribing English speech...")
model = whisper.load_model("base")
result = model.transcribe(temp_audio_path)

# ==== Step 3: Translate text and generate subtitles ====
print(f"🌐 Translating to {chosen_language.title()}...")
translator = Translator()
translated_segments = []
srt_file = pysrt.SubRipFile()
tts_text_full = ""

for idx, seg in enumerate(result["segments"]):
    start, end, text = seg["start"], seg["end"], seg["text"]
    translated_text = translator.translate(text, dest=lang_code).text
    translated_segments.append(translated_text)
    tts_text_full += translated_text + " "

    sub = pysrt.SubRipItem()
    sub.index = idx + 1
    sub.start.seconds = int(start)
    sub.end.seconds = int(end)
    sub.text = translated_text
    srt_file.append(sub)

srt_file.save(translated_srt_path, encoding='utf-8')

# ==== Step 4: Generate audio ====
if use_tts:
    print("🎤 Generating TTS audio...")
    tts = gTTS(text=tts_text_full, lang=lang_code)
    tts.save("tts_output.mp3")

    tts_audio = AudioSegment.from_file("tts_output.mp3")
    video_duration_ms = int(video.duration * 1000)
    tts_duration_ms = len(tts_audio)

    if tts_duration_ms < video_duration_ms:
        padding = AudioSegment.silent(duration=(video_duration_ms - tts_duration_ms))
        final_audio = tts_audio + padding
    else:
        final_audio = tts_audio[:video_duration_ms]

    final_audio += 6  # Optional volume boost
    final_audio.export(translated_audio_path, format="mp3")

    print("🎬 Merging audio with video...")
    translated_audio = AudioFileClip(translated_audio_path)
    final_video = video.set_audio(translated_audio)
    final_video.write_videofile(translated_video_path, codec='libx264', audio_codec='aac')
import os
import whisper
from moviepy.editor import VideoFileClip, AudioFileClip
from googletrans import Translator
from gtts import gTTS
from pydub import AudioSegment
import pysrt

# ==== CONFIGURATION ====
video_path = "/content/input.mp4"
translated_video_path = "translated_video.mp4"
translated_audio_path = "translated_audio.mp3"
translated_srt_path = "translated_subs.srt"
temp_audio_path = "temp_audio.wav"
chosen_language = "malayalam"  # Change to your target language

# ==== Language Support ====
language_codes = {
    "english": "en", "hindi": "hi", "bengali": "bn", "tamil": "ta", "telugu": "te", "marathi": "mr",
    "gujarati": "gu", "kannada": "kn", "malayalam": "ml", "punjabi": "pa", "urdu": "ur",
    "assamese": "as", "maithili": "mai", "santali": "sat", "bodo": "brx", "dogri": "doi",
    "kashmiri": "ks", "konkani": "gom", "sanskrit": "sa", "sindhi": "sd"
}
gtts_supported = {"hi", "bn", "ta", "te", "mr", "gu", "kn", "ml", "pa", "ur"}

lang_code = language_codes.get(chosen_language.lower(), "ml")
use_tts = lang_code in gtts_supported

# ==== Step 1: Extract audio from video ====
print("🔊 Extracting audio from video...")
video = VideoFileClip(video_path)
video.audio.write_audiofile(temp_audio_path)

# ==== Step 2: Transcribe English audio ====
print("🧠 Transcribing English speech...")
model = whisper.load_model("base")
result = model.transcribe(temp_audio_path)

# ==== Step 3: Translate text and generate subtitles ====
print(f"🌐 Translating to {chosen_language.title()}...")
translator = Translator()
translated_segments = []
srt_file = pysrt.SubRipFile()
tts_text_full = ""

for idx, seg in enumerate(result["segments"]):
    start, end, text = seg["start"], seg["end"], seg["text"]
    translated_text = translator.translate(text, dest=lang_code).text
    translated_segments.append(translated_text)
    tts_text_full += translated_text + " "

    sub = pysrt.SubRipItem()
    sub.index = idx + 1
    sub.start.seconds = int(start)
    sub.end.seconds = int(end)
    sub.text = translated_text
    srt_file.append(sub)

srt_file.save(translated_srt_path, encoding='utf-8')

# ==== Step 4: Generate audio ====
if use_tts:
    print("🎤 Generating TTS audio...")
    tts = gTTS(text=tts_text_full, lang=lang_code)
    tts.save("tts_output.mp3")

    tts_audio = AudioSegment.from_file("tts_output.mp3")
    video_duration_ms = int(video.duration * 1000)
    tts_duration_ms = len(tts_audio)

    if tts_duration_ms < video_duration_ms:
        padding = AudioSegment.silent(duration=(video_duration_ms - tts_duration_ms))
        final_audio = tts_audio + padding
    else:
        final_audio = tts_audio[:video_duration_ms]

    final_audio += 6  # Optional volume boost
    final_audio.export(translated_audio_path, format="mp3")

    print("🎬 Merging audio with video...")
    translated_audio = AudioFileClip(translated_audio_path)
    final_video = video.set_audio(translated_audio)
    final_video.write_videofile(translated_video_path, codec='libx264', audio_codec='aac')
    print("✅ Done! Translated video created.")
else:
    print(f"⚠️ Audio generation not supported for '{chosen_language}'")

  if event.key is 'enter':



🔊 Extracting audio from video...
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
🧠 Transcribing English speech...


100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 34.6MiB/s]
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



🌐 Translating to Malayalam...
🎤 Generating TTS audio...
🎬 Merging audio with video...
Moviepy - Building video translated_video.mp4.
MoviePy - Writing audio in translated_videoTEMP_MPY_wvf_snd.mp4


...

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



🌐 Translating to Malayalam...
🎤 Generating TTS audio...
🎬 Merging audio with video...
Moviepy - Building video translated_video.mp4.
MoviePy - Writing audio in translated_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video translated_video.mp4



t: 100%|█████████▉| 2740/2745 [01:19<00:00, 26.93it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/input.mp4, 1218900 bytes wanted but 0 bytes read,at frame 2741/2745, at time 91.37/91.49 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|█████████▉| 2744/2745 [01:19<00:00, 26.82it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/input.mp4, 1218900 bytes wanted but 0 bytes read,at frame 2743/2745, at time 91.43/91.49 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready translated_video.mp4
✅ Done! Translated video created.
